### 1

In [ ]:
import sklearn
import numpy as np
import os
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
from prettytable import PrettyTable
%matplotlib inline
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.4)
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
tqdm_notebook.get_lock().locks = []
# !pip install sweetviz
# import sweetviz as sv
import concurrent.futures
from copy import deepcopy
from functools import partial
from itertools import combinations
import random
from random import randint, uniform
import gc
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler,PowerTransformer, FunctionTransformer
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from itertools import combinations
from sklearn.impute import SimpleImputer
import xgboost as xg
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error,mean_squared_log_error, roc_auc_score, accuracy_score, f1_score, precision_recall_curve, log_loss
from sklearn.cluster import KMeans
!pip install yellowbrick
from yellowbrick.cluster import KElbowVisualizer
!pip install gap-stat
from gap_statistic.optimalK import OptimalK
from scipy import stats
import statsmodels.api as sm
from scipy.stats import ttest_ind
from scipy.stats import boxcox
import math
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.base import BaseEstimator, TransformerMixin
!pip install optuna
import optuna
!pip install cmaes
import cmaes
import xgboost as xgb
!pip install catboost
!pip install lightgbm --install-option=--gpu --install-option="--boost-root=C:/local/boost_1_69_0" --install-option="--boost-librarydir=C:/local/boost_1_69_0/lib64-msvc-14.1"
import lightgbm as lgb
!pip install category_encoders
from category_encoders import OneHotEncoder, OrdinalEncoder, CountEncoder, CatBoostEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier,ExtraTreesClassifier, AdaBoostClassifier, HistGradientBoostingRegressor
!pip install -U imbalanced-learn
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV, LogisticRegressionCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neural_network import MLPClassifier
from catboost import Pool
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")
pd.pandas.set_option('display.max_columns',None)

In [ ]:
table = PrettyTable()

table.field_names = ['Feature', 'Data Type', 'Train Missing %', 'Test Missing %',"Original Missing%"]
for column in train_copy.columns:
    data_type = str(train_copy[column].dtype)
    non_null_count_train= np.round(100-train_copy[column].count()/train_copy.shape[0]*100,1)
    if column!=target:
        non_null_count_test = np.round(100-test_copy[column].count()/test_copy.shape[0]*100,1)
    else:
        non_null_count_test="NA"
    non_null_count_orig= np.round(100-original_copy[column].count()/original_copy.shape[0]*100,1)
    table.add_row([column, data_type, non_null_count_train,non_null_count_test,non_null_count_orig])
print(table)

In [ ]:
def plot_pie_chart(data, title, ax):
    data_counts = data[target].value_counts()
    labels = data_counts.index
    sizes = data_counts.values
    colors = [ (0.2, 0.9, 0.6), 'crimson', (0.8, 0.5, 0.3)]
    explode = (0.1,0.05, 0)

    ax.pie(sizes,explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
    ax.axis('equal')
    ax.set_title(title)

fig, axes = plt.subplots(1, 2, figsize=(18, 6))  # Create three subplots in a row

plot_pie_chart(train_copy, "Train Churn Distribution", axes[0])
plot_pie_chart(original, "Original Churn Distribution", axes[1])

plt.tight_layout()
plt.show()

In [ ]:
cat_cols = [f for f in train.columns if (train[f].dtype != 'O' and train[f].nunique() <100) or (train[f].dtype == 'O' and f not in [target]) ]
custom_palette = (0.2, 0.9, 0.6), 'crimson', (0.8, 0.5, 0.3)
for col in cat_cols:
    contingency_table = pd.crosstab(train[col], train[target], normalize='index')
    sns.set(style="whitegrid")
    contingency_table.plot(kind="bar", stacked=True, color=custom_palette,figsize=(20, 4))
    plt.title(f"Percentage Distribution of Target across {col}")
    plt.xlabel(col)
    plt.ylabel("Percentage")
    plt.legend(title="Target Class")
    plt.show()

In [ ]:
target_map={
'Graduate':0,
'Enrolled':1,
'Dropout':2}
lgb_params = {
    'n_estimators': 50,
    'max_depth': 8,
    'learning_rate': 0.02,
    'subsample': 0.20,
    'colsample_bytree': 0.56,
    'reg_alpha': 0.25,
    'reg_lambda': 5e-08,
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'random_state': 42,
    'device':device.lower(),
    'verbose':-1
    }

def encode(y,target_map):
    '''
    To convert the outputs to numbers
    '''
    y=np.array(y)
    encoded_y=[target_map[f] for f in y]
    return encoded_y
def decode(y,target_map):
    '''To convert the predictions back to classes
    '''
    y=np.array(y)
    reverse_dict={v: k for k, v in target_map.items()}
    decoded_y=[reverse_dict[f] for f in y]
    return decoded_y
def min_max_scaler(train, test, column):
    '''
    Min Max just based on train might have an issue if test has extreme values, hence changing the denominator using overall min and max
    '''
    max_val=max(train[column].max(),test[column].max())
    min_val=min(train[column].min(),test[column].min())

    train[column]=(train[column]-min_val)/(max_val-min_val)
    test[column]=(test[column]-min_val)/(max_val-min_val)

    return train,test

hgbm_params={'learning_rate': 0.07590136944725749, 'max_depth': 6, 'min_samples_leaf': 12, 'max_leaf_nodes': 5}
lgb_params1 = {
            'n_estimators': 50,
            'max_depth': 6,
            "num_leaves": 16,
            'learning_rate': 0.002,
            'subsample': 0.7,
            'colsample_bytree': 0.8,
            #'reg_alpha': 0.25,
#             'reg_lambda': 5e-07,
            "boosting_type": "gbdt",
            "objective":'regression_l1',
            'metric': 'mean_absolute_error',
            'device': device.lower(),
            'random_state': 42,
            'verbose':-1
        }
def store_missing_rows(df, features):
    '''Function stores where missing values are located for given set of features'''
    missing_rows = {}

    for feature in features:
        missing_rows[feature] = df[df[feature].isnull()]

    return missing_rows
def fill_missing_numerical(train,test,target, max_iterations=10):
    '''Iterative Missing Imputer: Updates filled missing values iteratively using CatBoost Algorithm'''
    train_temp=train.copy()
    if target in train_temp.columns:
        train_temp=train_temp.drop(columns=target)


    df=pd.concat([train_temp,test],axis="rows")
    df=df.reset_index(drop=True)
    features=[ f for f in df.columns if df[f].isna().sum()>0]
    if len(features)>0:
        # Step 1: Store the instances with missing values in each feature
        missing_rows = store_missing_rows(df, features)

        # Step 2: Initially fill all missing values with "Missing"
        for f in features:
            df[f]=df[f].fillna(df[f].median())

        cat_features=[f for f in df.columns if not pd.api.types.is_numeric_dtype(df[f])]
        dictionary = {feature: [] for feature in features}

        for iteration in tqdm(range(max_iterations), desc="Iterations"):
            for feature in features:
#                 print(feature)
                # Skip features with no missing values
                rows_miss = missing_rows[feature].index
                replace_dict={}
                rev_replace_dict={}
                for col in  cat_features:
                    df[col]=df[col].astype(str)
                    int_cat=dict(zip(df[col].unique(),np.arange(0, df[col].nunique())))
                    rev_int_cat=dict(zip(np.arange(0, df[col].nunique()), df[col].unique()))
                    df[col]=df[col].replace(int_cat)

                    replace_dict[col]=int_cat
                    rev_replace_dict[col]=rev_int_cat

                missing_temp = df.loc[rows_miss].copy()
                non_missing_temp = df.drop(index=rows_miss).copy()
                y_pred_prev=missing_temp[feature]
                missing_temp = missing_temp.drop(columns=[feature])


                # Step 3: Use the remaining features to predict missing values using Random Forests
                X_train = non_missing_temp.drop(columns=[feature])
                y_train = non_missing_temp[[feature]]

                model1 =HistGradientBoostingRegressor(**hgbm_params, max_iter=100, loss="absolute_error", n_iter_no_change=50,random_state=42)
                model1.fit(X_train, y_train)

                model2 = lgb.LGBMRegressor(**lgb_params1)
                model2.fit(X_train, y_train)

                # Step 4: Predict missing values for the feature and update all N features
                y_pred = (np.array(model1.predict(missing_temp))+np.array(model2.predict(missing_temp)))/2
                df.loc[rows_miss, feature] = y_pred
#                 error_minimize=rmse(y_pred,y_pred_prev) #mean_squared_error
                error_minimize=mean_squared_error(y_pred,y_pred_prev) #mean_squared_error
                dictionary[feature].append(error_minimize)  # Append the error_minimize value

                for col in  cat_features:
                    df[col]=df[col].replace(rev_int_cat)


        for feature, values in dictionary.items():
            values=np.array(values)/sum(values)
            iterations = range(1, len(values) + 1)  # x-axis values (iterations)
            plt.plot(iterations, values, label=feature)  # plot the values
            plt.xlabel('Iterations')
            plt.ylabel('RMSE')
            plt.title('Minimization of RMSE with iterations')
            plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.show()
        train[features] = np.array(df.iloc[:train.shape[0]][features])
        test[features] = np.array(df.iloc[train.shape[0]:][features])

    return train,test

In [ ]:
cat_cols = [f for f in test.columns if (train[f].dtype != 'O' and train[f].nunique()<100 and train[f].nunique()>2) or (train[f].dtype == 'O') ]
print(train[cat_cols].nunique())

In [ ]:
def nearest_val(target):
    return min(common, key=lambda x: abs(x - target))

cat_cols_updated=[]
for col in cat_cols:
    uncommon=list((set(test[col].unique())| set(train[col].unique()))-(set(test[col].unique())& set(train[col].unique())))

    if train[col].dtype!="O":
        train[f"{col}_cat"]=train[col]
        test[f"{col}_cat"]=test[col]
        cat_cols_updated.append(f"{col}_cat")
        if uncommon:
            common=list(set(test[col].unique())& set(train[col].unique()))
            train[f"{col}_cat"]=train[col].apply(lambda x: np.nan if x in uncommon else x)#train[col].apply(nearest_val)
            test[f"{col}_cat"]=test[col].apply(lambda x: np.nan if x in uncommon else x)#test[col].apply(nearest_val)
    else:
        cat_cols_updated.append(col)
        train[col]=train[col].astype(str)+"_"+col
        test[col]=test[col].astype(str)+"_"+col
        uncommon=list((set(test[col].unique())| set(train[col].unique()))-(set(test[col].unique())& set(train[col].unique())))
        train[col]=train[col].apply(lambda x: np.nan if x in uncommon else x)
        test[col]=test[col].apply(lambda x: np.nan if x in uncommon else x)

print(train[cat_cols_updated].nunique())

In [ ]:
global overall_best_score
overall_best_score = 0
def OHE(train_df,test_df,cols,target):
    '''
    Function for one hot encoding, it first combines the data so that no category is missed and
    the category with least frequency can be dropped because of redundancy
    '''
    combined = pd.concat([train_df, test_df], axis=0)
    for col in cols:
        one_hot = pd.get_dummies(combined[col]).astype(int)
        counts = combined[col].value_counts()
        min_count_category = counts.idxmin()
        one_hot = one_hot.drop(min_count_category, axis=1)
        one_hot.columns=[str(f)+col+"_OHE" for f in one_hot.columns]
        combined = pd.concat([combined, one_hot], axis="columns")
        combined = combined.loc[:, ~combined.columns.duplicated()]

    # split back to train and test dataframes
    train_ohe = combined[:len(train_df)]
    test_ohe = combined[len(train_df):]
    test_ohe.reset_index(inplace=True,drop=True)
    test_ohe.drop(columns=[target],inplace=True)
    return train_ohe, test_ohe

def high_freq_ohe(train, test, extra_cols, target, n_limit=50):
    '''
    If you wish to apply one hot encoding on a feature with so many unique values, then this can be applied,
    where it takes a maximum of n categories and drops the rest of them treating as rare categories
    '''
    train_copy=train.copy()
    test_copy=test.copy()
    ohe_cols=[]
    for col in extra_cols:
        dict1=train_copy[col].value_counts().to_dict()
        ordered=dict(sorted(dict1.items(), key=lambda x: x[1], reverse=True))
        rare_keys=list([*ordered.keys()][n_limit:])
#         ext_keys=[f[0] for f in ordered.items() if f[1]<50]
        rare_key_map=dict(zip(rare_keys, np.full(len(rare_keys),9999)))

        train_copy[col]=train_copy[col].replace(rare_key_map)
        test_copy[col]=test_copy[col].replace(rare_key_map)
    train_copy, test_copy = OHE(train_copy, test_copy, extra_cols, target)
    drop_cols=[f for f in train_copy.columns if "9999" in f or train_copy[f].nunique()==1]
    train_copy=train_copy.drop(columns=drop_cols)
    test_copy=test_copy.drop(columns=drop_cols)

    return train_copy, test_copy

def cat_encoding(train, test,cat_cols_updated, target):
    global overall_best_score
    global overall_best_col
    table = PrettyTable()
    table.field_names = ['Feature', 'Encoded Features', 'Accuracy Score']
    train_copy=train.copy()
    test_copy=test.copy()
    train_dum = train.copy()
    for feature in cat_cols_updated:
#         print(feature)
#         cat_labels = train_dum.groupby([feature])[target].mean().sort_values().index
#         cat_labels2 = {k: i for i, k in enumerate(cat_labels, 0)}
#         train_copy[feature + "_target"] = train[feature].map(cat_labels2)
#         test_copy[feature + "_target"] = test[feature].map(cat_labels2)

        dic = train[feature].value_counts().to_dict()
        train_copy[feature + "_count"] =train[feature].map(dic)
        test_copy[feature + "_count"] = test[feature].map(dic)

        dic2=train[feature].value_counts().to_dict()
#         list1=np.arange(len(dic2.values()),0,-1) # Higher rank for high count
        list1=np.arange(len(dic2.values())) # Higher rank for low count
        dic3=dict(zip(list(dic2.keys()),list1))

        train_copy[feature+"_count_label"]=train[feature].replace(dic3).astype(float)
        test_copy[feature+"_count_label"]=test[feature].replace(dic3).astype(float)

        temp_cols = [ feature + "_count", feature + "_count_label"]#,feature + "_target"

        train_copy[feature]=train_copy[feature].astype(str)+"_"+feature
        test_copy[feature]=test_copy[feature].astype(str)+"_"+feature

        if train_copy[feature].nunique()<=10:
            train_copy[feature]=train_copy[feature].astype(str)+"_"+feature
            test_copy[feature]=test_copy[feature].astype(str)+"_"+feature
            train_copy, test_copy = OHE(train_copy, test_copy, [feature], target)

        else:
            train_copy,test_copy=high_freq_ohe(train_copy,test_copy,[feature], target, n_limit=10)

        train_copy=train_copy.drop(columns=[feature])
        test_copy=test_copy.drop(columns=[feature])

        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        ll_scores = []

        for f in temp_cols:
            X = train_copy[[f]].values
            y = train_copy[target].values

            log_loss_score = []
            for train_idx, val_idx in kf.split(X, y):
                X_train, y_train = X[train_idx], y[train_idx]
                x_val, y_val = X[val_idx], y[val_idx]
                model =  lgb.LGBMClassifier(**lgb_params)

                model.fit(X_train, encode(y_train, target_map))
                y_pred = model.predict(x_val)
                log_loss_score.append(accuracy_score(encode(y_val, target_map),y_pred))

            ll_scores.append((f, np.mean(log_loss_score)))
            if overall_best_score < np.mean(log_loss_score):
                overall_best_score = np.mean(log_loss_score)
                overall_best_col = f
        best_col, best_loss = sorted(ll_scores, key=lambda x: x[1], reverse=True)[0]

        corr = train_copy[temp_cols].corr(method='pearson')
        corr_with_best_col = corr[best_col]
        cols_to_drop = [f for f in temp_cols if corr_with_best_col[f] > 0.5 and f != best_col]
        final_selection = [f for f in temp_cols if f not in cols_to_drop]
        if cols_to_drop:
            train_copy = train_copy.drop(columns=cols_to_drop)
            test_copy = test_copy.drop(columns=cols_to_drop)

        table.add_row([feature, best_col, best_loss])

    print(table)
    print("overall best CV score: ", overall_best_score)
    return train_copy, test_copy

train, test= cat_encoding(train, test,cat_cols_updated, target)

In [ ]:
def better_features(train, test, target, cols, best_score):
    new_cols = []
    skf = KFold(n_splits=5, shuffle=True, random_state=42)  # Stratified k-fold object
    best_list=[]
    for i in tqdm(range(len(cols)), desc='Generating Columns'):
        col1 = cols[i]
        temp_df = pd.DataFrame()  # Temporary dataframe to store the generated columns
        temp_df_test = pd.DataFrame()  # Temporary dataframe for test data

        for j in range(i+1, len(cols)):
            col2 = cols[j]
            # Multiply
            temp_df[col1 + '*' + col2] = train[col1] * train[col2]
            temp_df_test[col1 + '*' + col2] = test[col1] * test[col2]

            # Divide (col1 / col2)
            temp_df[col1 + '/' + col2] = train[col1] / (train[col2] + 1e-5)
            temp_df_test[col1 + '/' + col2] = test[col1] / (test[col2] + 1e-5)

            # Divide (col2 / col1)
            temp_df[col2 + '/' + col1] = train[col2] / (train[col1] + 1e-5)
            temp_df_test[col2 + '/' + col1] = test[col2] / (test[col1] + 1e-5)

            # Subtract
            temp_df[col1 + '-' + col2] = train[col1] - train[col2]
            temp_df_test[col1 + '-' + col2] = test[col1] - test[col2]

            # Add
            temp_df[col1 + '+' + col2] = train[col1] + train[col2]
            temp_df_test[col1 + '+' + col2] = test[col1] + test[col2]

        SCORES = []
        for column in temp_df.columns:
            scores = []
            for train_index, val_index in skf.split(train, train[target]):
                X_train, X_val = temp_df[column].iloc[train_index].values.reshape(-1, 1), temp_df[column].iloc[val_index].values.reshape(-1, 1)
                y_train, y_val = train[target].iloc[train_index], train[target].iloc[val_index]
                model =lgb.LGBMClassifier(**lgb_params)
                model.fit(X_train, encode(y_train, target_map))
                y_pred = model.predict(X_val)
                score = accuracy_score(encode(y_val, target_map),y_pred)
                scores.append(score)
            mean_score = np.mean(scores)
            SCORES.append((column, mean_score))

        if SCORES:
            best_col, best_acc = sorted(SCORES, key=lambda x: x[1],reverse=True)[0]
            corr_with_other_cols = train.drop([target] + new_cols, axis=1).corrwith(temp_df[best_col])
            if (corr_with_other_cols.abs().max() < 0.9 or best_acc > best_score) and corr_with_other_cols.abs().max() !=1 :
                train[best_col] = temp_df[best_col]
                test[best_col] = temp_df_test[best_col]
                new_cols.append(best_col)
                print(f"Added column '{best_col}' with Accuracy Score: {best_acc:.4f} & Correlation {corr_with_other_cols.abs().max():.4f}")

    return train, test, new_cols


In [ ]:
selected_features=[f for f in test.columns if train[f].nunique()>2]
len(selected_features)

# train, test,new_cols=better_features(train, test, target, selected_features, overall_best_score)
# new_cols

In [ ]:
new_cols=['Curricular units 2nd sem (approved)/Marital status',
 'Application mode+Curricular units 2nd sem (approved)_cat_count',
 'Application order-Curricular units 2nd sem (approved)_cat_count',
 'Course/Curricular units 2nd sem (approved)',
 'Previous qualification+Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 2nd sem (approved)/Previous qualification (grade)',
 "Mother's qualification-Curricular units 2nd sem (approved)_cat_count",
 "Father's qualification+Curricular units 2nd sem (approved)_cat_count",
 "Mother's occupation-Curricular units 2nd sem (approved)_cat_count",
 "Father's occupation+Curricular units 2nd sem (approved)_cat_count",
 'Admission grade*Curricular units 2nd sem (approved)',
 'Age at enrollment-Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 1st sem (credited)+Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 1st sem (enrolled)/Curricular units 2nd sem (approved)',
 'Curricular units 1st sem (evaluations)+Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 1st sem (approved)/Curricular units 2nd sem (approved)',
 'Curricular units 1st sem (grade)*Curricular units 2nd sem (approved)',
 'Curricular units 1st sem (without evaluations)-Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 2nd sem (credited)+Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 2nd sem (approved)/Curricular units 2nd sem (enrolled)',
 'Curricular units 2nd sem (evaluations)-Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 2nd sem (approved)/Curricular units 1st sem (approved)_cat_count',
 'Curricular units 2nd sem (grade)-Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 2nd sem (without evaluations)-Curricular units 2nd sem (approved)_cat_count',
 'Unemployment rate-Curricular units 2nd sem (approved)_cat_count',
 'Inflation rate-Curricular units 2nd sem (approved)_cat_count',
 'GDP+Curricular units 2nd sem (approved)_cat_count',
 'Marital status_cat_count*Curricular units 2nd sem (approved)_cat_count_label',
 'Marital status_cat_count_label-Curricular units 2nd sem (approved)_cat_count',
 'Application mode_cat_count*Curricular units 2nd sem (approved)_cat_count_label',
 'Application mode_cat_count_label+Curricular units 2nd sem (approved)_cat_count',
 'Application order_cat_count*Curricular units 2nd sem (approved)_cat_count_label',
 'Application order_cat_count_label-Curricular units 2nd sem (approved)_cat_count',
 'Course_cat_count+Curricular units 2nd sem (approved)_cat_count',
 'Course_cat_count_label+Curricular units 2nd sem (approved)_cat_count',
 'Previous qualification_cat_count*Curricular units 2nd sem (approved)_cat_count_label',
 'Previous qualification_cat_count_label-Curricular units 2nd sem (approved)_cat_count',
 'Nacionality_cat_count/Curricular units 2nd sem (approved)_cat_count',
 "Mother's qualification_cat_count/Curricular units 2nd sem (approved)_cat_count_label",
 "Mother's qualification_cat_count_label-Curricular units 2nd sem (approved)_cat_count",
 "Father's qualification_cat_count*Curricular units 2nd sem (approved)_cat_count_label",
 "Father's qualification_cat_count_label-Curricular units 2nd sem (approved)_cat_count",
 "Mother's occupation_cat_count*Curricular units 2nd sem (approved)_cat_count_label",
 "Mother's occupation_cat_count_label-Curricular units 2nd sem (approved)_cat_count",
 "Father's occupation_cat_count*Curricular units 2nd sem (approved)_cat_count_label",
 "Father's occupation_cat_count_label+Curricular units 2nd sem (approved)_cat_count",
 'Curricular units 2nd sem (approved)_cat_count_label/Age at enrollment_cat_count',
 'Age at enrollment_cat_count_label-Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 1st sem (credited)_cat_count+Curricular units 2nd sem (approved)_cat_count_label',
 'Curricular units 1st sem (credited)_cat_count_label+Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 1st sem (enrolled)_cat_count+Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 1st sem (enrolled)_cat_count_label+Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 1st sem (evaluations)_cat_count*Curricular units 2nd sem (approved)_cat_count_label',
 'Curricular units 1st sem (evaluations)_cat_count_label+Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 1st sem (approved)_cat_count-Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 1st sem (approved)_cat_count_label-Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 1st sem (without evaluations)_cat_count+Curricular units 2nd sem (approved)_cat_count_label',
 'Curricular units 1st sem (without evaluations)_cat_count_label-Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 2nd sem (credited)_cat_count/Curricular units 2nd sem (approved)_cat_count_label',
 'Curricular units 2nd sem (credited)_cat_count_label+Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 2nd sem (enrolled)_cat_count+Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 2nd sem (approved)_cat_count/Curricular units 2nd sem (enrolled)_cat_count_label',
 'Curricular units 2nd sem (evaluations)_cat_count*Curricular units 2nd sem (approved)_cat_count_label',
 'Curricular units 2nd sem (evaluations)_cat_count_label-Curricular units 2nd sem (approved)_cat_count',
 'Curricular units 2nd sem (approved)_cat_count-Inflation rate_cat_count_label',
 'Inflation rate_cat_count/Curricular units 2nd sem (without evaluations)_cat_count_label',
 'Unemployment rate_cat_count_label*Inflation rate_cat_count']


In [ ]:
def apply_arithmetic_operations(train_df, test_df, expressions_list):
    for expression in expressions_list:
        if expression not in train_df.columns:
            # Split the expression based on operators (+, -, *, /)
            parts = expression.split('+') if '+' in expression else \
                    expression.split('-') if '-' in expression else \
                    expression.split('*') if '*' in expression else \
                    expression.split('/')

            # Get the DataFrame column names involved in the operation
            cols = [col for col in parts]

            # Perform the corresponding arithmetic operation based on the operator in the expression
            if cols[0] in train_df.columns and cols[1] in train_df.columns:
                if '+' in expression:
                    train_df[expression] = train_df[cols[0]] + train_df[cols[1]]
                    test_df[expression] = test_df[cols[0]] + test_df[cols[1]]
                elif '-' in expression:
                    train_df[expression] = train_df[cols[0]] - train_df[cols[1]]
                    test_df[expression] = test_df[cols[0]] - test_df[cols[1]]
                elif '*' in expression:
                    train_df[expression] = train_df[cols[0]] * train_df[cols[1]]
                    test_df[expression] = test_df[cols[0]] * test_df[cols[1]]
                elif '/' in expression:
                    train_df[expression] = train_df[cols[0]] / (train_df[cols[1]]+1e-5)
                    test_df[expression] = test_df[cols[0]] /( test_df[cols[1]]+1e-5)

    return train_df, test_df

train, test = apply_arithmetic_operations(train, test, new_cols)


In [ ]:
final_features=[f for f in train.columns if f not in [target]]
final_features=[*set(final_features)]

sc=StandardScaler()

train_scaled=train.copy()
test_scaled=test.copy()
train_scaled[final_features]=sc.fit_transform(train[final_features])
test_scaled[final_features]=sc.transform(test[final_features])

In [ ]:
# train_cop, test_cop=fill_missing_numerical(train_cop, test_cop,target, max_iterations=2)

missing_cols=[f for f in test_cop.columns if train_cop[f].isna().sum()+test_cop[f].isna().sum()>0]

for f in missing_cols:
    train_cop[f]=train_cop[f].fillna(train_cop[f].median())
    test_cop[f]=test_cop[f].fillna(test_cop[f].median())

In [ ]:
X_train = train_cop.drop(columns=[target])
y_train = train[target]

X_test = test_cop.copy()

print(X_train.shape, X_test.shape)

In [ ]:
def get_most_important_features(X_train, y_train, n,model_input):
    xgb_params = {
            'n_estimators': 200,
            'learning_rate': 0.05,
            'max_depth': 4,
            'subsample': 0.8,
            'colsample_bytree': 0.1,
            'n_jobs': -1,
            'eval_metric': 'mlogloss',
            'objective': 'multi:softprob',
            'tree_method': 'hist',
            'verbosity': 0,
            'random_state': 42,
        }
    if device.lower() == 'gpu':
            xgb_params['tree_method'] = 'gpu_hist'
            xgb_params['predictor'] = 'gpu_predictor'
    lgb_params = {
            'n_estimators': 200,
            'max_depth': 7,
            'learning_rate': 0.05,
            'subsample': 0.20,
            'colsample_bytree': 0.56,
            'reg_alpha': 0.25,
            'reg_lambda': 5e-08,
            'objective': 'multiclass',
            'metric': 'multi_logloss',
            'boosting_type': 'gbdt',
            'random_state': 42,
            'device': device.lower(),
            'verbose':-1
        }
    cb_params = {
            'iterations': 200,
            'depth': 7,
            'learning_rate': 0.1,
            'l2_leaf_reg': 0.7,
            'random_strength': 0.2,
            'max_bin': 200,
            'od_wait': 65,
            'one_hot_max_size': 70,
            'grow_policy': 'Depthwise',
            'bootstrap_type': 'Bayesian',
            'od_type': 'Iter',
            'eval_metric': 'MultiClass',
            'loss_function': 'MultiClass',
            'task_type': device.upper(),
            'random_state': 42,
        }
    if 'xgb' in model_input:
        model = xgb.XGBClassifier(**xgb_params)
    elif 'cat' in model_input:
        model=CatBoostClassifier(**cb_params)
    else:
        model=lgb.LGBMClassifier(**lgb_params)

    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    f1_scores = []
    feature_importances_list = []

    for train_idx, val_idx in kfold.split(X_train):
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

        if 'lgb' in model_input:
            model.fit(X_train_fold, encode(y_train_fold,target_map))
        else:
            model.fit(X_train_fold,encode(y_train_fold,target_map),verbose=False)

        y_pred = model.predict(X_val_fold)
        f1_scores.append(accuracy_score(encode(y_val_fold,target_map), y_pred))
        feature_importances = model.feature_importances_
        feature_importances_list.append(feature_importances)

    avg_f1 = np.mean(f1_scores)
    avg_feature_importances = np.mean(feature_importances_list, axis=0)

    feature_importance_list = [(X_train.columns[i], importance) for i, importance in enumerate(avg_feature_importances)]
    sorted_features = sorted(feature_importance_list, key=lambda x: x[1], reverse=True)
    top_n_features = [feature[0] for feature in sorted_features[:n]]

    display_features=top_n_features[:12]

    sns.set_palette([(0.8, 0.5, 0.3)])
    plt.figure(figsize=(10, 12))
    plt.barh(range(len(display_features)), [avg_feature_importances[X_train.columns.get_loc(feature)] for feature in display_features])
    plt.yticks(range(len(display_features)), display_features, fontsize=12)
    plt.xlabel('Average Feature Importance', fontsize=14)
    plt.ylabel('Features', fontsize=10)
    plt.title(f'Top {len(display_features)} of {n} Feature Importances with best Accuracy score {avg_f1}', fontsize=16)
    plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature on top
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.xticks(fontsize=8)
    plt.yticks(fontsize=8)

    # Add data labels on the bars
    for index, value in enumerate([avg_feature_importances[X_train.columns.get_loc(feature)] for feature in display_features]):
        plt.text(value + 0.005, index, f'{value:.3f}', fontsize=12, va='center')

    plt.tight_layout()
    plt.show()

    return top_n_features

In [ ]:
n_imp_features_cat=get_most_important_features(X_train.reset_index(drop=True), y_train,150, 'cat')
n_imp_features_xgb=get_most_important_features(X_train.reset_index(drop=True), y_train,150, 'xgb')
n_imp_features_lgbm=get_most_important_features(X_train.reset_index(drop=True), y_train,150, 'lgbm')

In [ ]:
n_imp_features=[*set(n_imp_features_xgb+n_imp_features_lgbm+n_imp_features_cat)]
print(f"{len(n_imp_features)} features have been selected from three algorithms for the final model")

X_train=X_train[n_imp_features]
X_test=X_test[n_imp_features]

In [ ]:
classes = np.unique(y_train)  # Get unique class labels
class_to_index = {cls: idx for idx, cls in enumerate(classes)}
y_train_numeric = np.array([class_to_index[cls] for cls in y_train])

class_counts = np.bincount(y_train_numeric)

total_samples = len(y_train_numeric)

class_weights = total_samples / (len(classes) * class_counts)

class_weights_dict = {target_map[cls]: weight for cls, weight in zip(classes, class_weights)}

print("Class counts:", class_counts)
print("Total samples:", total_samples)
print("Class weights:", class_weights)
print("Class weights dictionary:", class_weights_dict)

In [ ]:
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LeakyReLU, PReLU, ELU
from keras.layers import Dropout

from keras.utils import to_categorical

gpus = tensorflow.config.list_physical_devices('GPU')
if gpus:
    print("GPU is available")
else:
    print("GPU is not available")


def optimizer():
    sgd=tensorflow.keras.optimizers.SGD(learning_rate=0.005, momentum=0.5, nesterov=True)
    rms = tensorflow.keras.optimizers.RMSprop()
    nadam=tensorflow.keras.optimizers.Nadam(
        learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam"
    )
    adam=tensorflow.keras.optimizers.Adam()
    adamW = keras.optimizers.AdamW(learning_rate=0.002,weight_decay=0.001, beta_1=0.9, beta_2=0.999)

    return sgd,rms,nadam, adamW,adam


lrelu = lambda x: tensorflow.keras.activations.relu(x, alpha=0.1)

from tensorflow.keras import backend as K

def root_mean_squared_log_error(y_true, y_pred):
    '''
    Compute RMSLE between actuals & predictions
    '''
    return K.sqrt(K.mean(K.square(K.log(abs(y_pred+1)) - K.log(abs(y_true+1)))))

def root_mean_squared_error(y_true, y_pred):
    '''
    Compute RMSE between actuals & predictions
    '''
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
def init_ann1(num_classes, input_dim):
    '''
    Initialize the artificial neural network (ANN) for multiclass classification
    '''

    sgd, rms, nadam, adamW, adam = optimizer()

    ann = Sequential()
    ann.add(Dense(288, input_dim=input_dim, kernel_initializer='he_uniform', activation='relu'))
    ann.add(Dropout(0.4))
    ann.add(Dense(80, kernel_initializer='he_uniform', activation='relu'))
    ann.add(Dropout(0.1))
    ann.add(Dense(32, kernel_initializer='he_uniform', activation='relu'))
    ann.add(Dropout(0.0))

    # Change the output layer to match the number of classes
    ann.add(Dense(num_classes, kernel_initializer='he_uniform', activation='softmax'))

    # Compile the model with categorical_crossentropy loss and accuracy metric
    ann.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])

    return ann

def init_ann2(num_classes, input_dim):
    sgd,rms,nadam, adamW, adam=optimizer()
    ann2 = Sequential()
    ann2.add(Dense(128, input_dim=X_test.shape[1], kernel_initializer='he_uniform', activation='relu'))
    ann2.add(Dropout(0.3))
    ann2.add(Dense(52,  kernel_initializer='he_uniform', activation='relu'))
    ann2.add(Dropout(0.1))
    ann2.add(Dense(12,  kernel_initializer='he_uniform', activation='relu'))
    ann2.add(Dropout(0.2))
#     ann2.add(Dense(16,  kernel_initializer='he_uniform', activation='relu'))
#     ann2.add(Dropout(0.1))

    ann2.add(Dense(num_classes, kernel_initializer='he_uniform', activation='softmax'))
    ann2.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    return ann2

def init_ann3(num_classes, input_dim):
    sgd,rms,nadam, adamW, adam=optimizer()
    ann3 = Sequential()
    ann3.add(Dense(384, input_dim=X_test.shape[1], kernel_initializer='he_uniform', activation='relu'))
    ann3.add(Dropout(0.3))
    ann3.add(Dense(84,  kernel_initializer='he_uniform', activation='relu'))
    ann3.add(Dropout(0.0))
#     ann3.add(Dense(112,  kernel_initializer='he_uniform', activation='relu'))
#     ann3.add(Dropout(0.1))
#     ann3.add(Dense(4,  kernel_initializer='he_uniform', activation='relu'))
#     ann3.add(Dropout(0.1))

    ann3.add(Dense(num_classes, kernel_initializer='he_uniform', activation='softmax'))
    ann3.compile(loss='categorical_crossentropy', optimizer=adamW, metrics=['accuracy'])

    return ann3

def init_ann4(num_classes, input_dim):
    sgd,rms,nadam, adamW, adam=optimizer()
    ann4 = Sequential()
    ann4.add(Dense(128, input_dim=X_test.shape[1], kernel_initializer='he_uniform', activation='relu'))
    ann4.add(Dropout(0.5))
    ann4.add(Dense(16,  kernel_initializer='he_uniform', activation='relu'))
    ann4.add(Dropout(0.3))
    # ann4.add(Dense(32,  kernel_initializer='he_uniform', activation='relu'))
    # ann4.add(Dropout(0.3))
    ann4.add(Dense(4,  kernel_initializer='he_uniform', activation='relu'))
    ann4.add(Dropout(0.1))

    ann4.add(Dense(num_classes, kernel_initializer='he_uniform', activation='softmax'))
    ann4.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return ann4

def init_ann5(num_classes, input_dim):
    sgd,rms,nadam, adamW, adam=optimizer()
    ann5 = Sequential()
    ann5.add(Dense(96, input_dim=X_test.shape[1], kernel_initializer='he_uniform', activation='relu'))
    ann5.add(Dropout(0.4))
    ann5.add(Dense(180,  kernel_initializer='he_uniform', activation='relu'))
#     ann5.add(Dropout(0.1))

    ann5.add(Dense(num_classes, kernel_initializer='he_uniform', activation='softmax'))
    ann5.compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])
    return ann5

In [ ]:
class Splitter:
    def __init__(self, test_size=0.2, kfold=True, n_splits=5):
        self.test_size = test_size
        self.kfold = kfold
        self.n_splits = n_splits

    def split_data(self, X, y, random_state_list):
        if self.kfold:
            for random_state in random_state_list:
                kf = StratifiedKFold(n_splits=self.n_splits, random_state=random_state, shuffle=True)
                for train_index, val_index in kf.split(X, y):
                    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
                    yield X_train, X_val, y_train, y_val

class Classifier:
    def __init__(self, n_estimators=100, device="cpu", random_state=0):
        self.n_estimators = n_estimators
        self.device = device
        self.random_state = random_state
        self.models = self._define_model()
        self.len_models = len(self.models)

    def _define_model(self):

        xgb_params = {
            'n_estimators': self.n_estimators,
            'learning_rate': 0.05,
            'max_depth': 4,
            'subsample': 0.8,
            'colsample_bytree': 0.1,
            'n_jobs': -1,
            'eval_metric': 'merror',
            'objective': 'multi_logloss',
            'tree_method': 'hist',
            'verbosity': 0,
            'random_state': self.random_state,
            'class_weight':class_weights_dict,
        }
        if self.device == 'gpu':
            xgb_params['tree_method'] = 'gpu_hist'
            xgb_params['predictor'] = 'gpu_predictor'

        xgb_params2 = {
            'n_estimators': self.n_estimators,
            'gamma': 0.2785290882258914,
            'max_depth': 10,
            'subsample': 0.32478148672785395,
             'min_child_weight': 9,
            'colsample_bytree': 0.4867041443764002,
            'learning_rate': 0.0521164563894605,
            'reg_lambda': 0.0006976301054985199,
            'reg_alpha': 0.37133730627394207,
            'n_jobs': -1,
            'eval_metric': 'merror',
            'objective': 'multi_logloss',
            'tree_method': 'hist',
            'verbosity': 0,
            'random_state': self.random_state,
            'class_weight':class_weights_dict,
        }
        if self.device == 'gpu':
            xgb_params2['tree_method'] = 'gpu_hist'
            xgb_params2['predictor'] = 'gpu_predictor'


        xgb_params3 = {
            'n_estimators': self.n_estimators,
            'gamma': 0.2785290882258914,
            'max_depth': 10,
            'subsample': 0.32478148672785395,
             'min_child_weight': 9,
            'colsample_bytree': 0.4867041443764002,
            'learning_rate': 0.0521164563894605,
            'reg_lambda': 0.0006976301054985199,
            'reg_alpha': 0.37133730627394207,
            'n_jobs': -1,
            'eval_metric': 'merror',
            'objective': 'multi_logloss',
            'tree_method': 'hist',
            'verbosity': 0,
            'device': 'cuda',
            'booster': 'gbtree',
            'random_state': self.random_state,
        }

        xgb_params4=xgb_params.copy()
        xgb_params4['subsample']= 0.7893839050813315
        xgb_params4['max_depth']=5
        xgb_params4['learning_rate']=0.161
        xgb_params4['colsample_bytree']=0.2425879035225036

        xgb_params5=xgb_params.copy()
#         xgb_params5['subsample']= 0.8380895263029425
#         xgb_params5['max_depth']=4
#         xgb_params5['learning_rate']=0.097
#         xgb_params5['colsample_bytree']=0.24119317177396393
        xgb_params5['objective']="multi:softprob"

        lgb_params = {
            'n_estimators': 1024,
            'max_depth': 11,
            'min_samples_leaf': 26,
            'subsample': 0.9118977524246277,
            'learning_rate': 0.028851311126998066,
            'lambda_l1': 4.1605396291792305,
            'lambda_l2': 2.6356292583061972e-05,
            'colsample_bytree': 0.2060085083494746,
#             'max_bin':245,
            'objective': 'multiclass',
            'metric': 'multi_logloss',
            'boosting_type': 'gbdt',
            'device': self.device,
            'random_state': self.random_state,
#             'class_weight':class_weights_dict,
            'verbose':-1
        }

        lgb_params2 = {
            'n_estimators': 1000,
            'max_depth': 6,
            'subsample': 0.7429283449560508,
            'learning_rate': 0.049029736097283075,
            'lambda_l1': 8.921656056016416e-05,
            'lambda_l2': 0.0017743761809582512,
            'colsample_bytree': 0.39212030104410694,
            'objective': 'multiclass',
            'metric': 'multi_error',
            'boosting_type': 'gbdt',
            'device': self.device,
            'random_state': self.random_state,
            'verbose':-1
        }

        lgb_params3 = {
            'n_estimators': 1000,
            'max_depth': 9,
            'subsample': 0.5396835462409598,
            'learning_rate': 0.049047285930431804,
            'lambda_l1': 1.7490663112850725e-08,
            'lambda_l2': 3.837216667252457,
            'colsample_bytree': 0.3193726625999373,
            'objective': 'multiclass',
            'metric': 'multi_error',
            'boosting_type': 'gbdt',
            'device': self.device,
            'random_state': self.random_state,
            'verbose':-1
        }
        lgb_params4=lgb_params2.copy()
        lgb_params4['subsample']=0.9
        lgb_params4['reg_lambda']=0.8762427869944874
        lgb_params4['reg_alpha']=0.31889688644219183
        lgb_params4['max_depth']=9
        lgb_params4['learning_rate']=0.107
        lgb_params4['colsample_bytree']=0.1

        lgb_params5=lgb_params2.copy()
        lgb_params5['subsample']=0.9
        lgb_params5['reg_lambda']=0.5119821036432592
        lgb_params5['reg_alpha']=0.8978283976480275
        lgb_params5['max_depth']=11
        lgb_params5['learning_rate']=0.081
        lgb_params5['colsample_bytree']=0.1
        cb_params = {
            'iterations': self.n_estimators,
            'depth': 6,
            'learning_rate': 0.05,
            'l2_leaf_reg': 0.7,
            'random_strength': 0.2,
            'max_bin': 200,
            'od_wait': 65,
            'one_hot_max_size': 70,
            'grow_policy': 'Depthwise',
            'bootstrap_type': 'Bayesian',
            'od_type': 'Iter',
            'eval_metric': 'Accuracy',
            'loss_function': 'MultiClass',
            'task_type': self.device.upper(),
            'random_state': self.random_state,
            'verbose':-1

        }
        cb_sym_params = cb_params.copy()
        cb_sym_params['grow_policy'] = 'SymmetricTree'
        cb_loss_params = cb_params.copy()
        cb_loss_params['grow_policy'] = 'Lossguide'

        cb_params2=  cb_params.copy()
        cb_params2['learning_rate']=0.019000000000000003
        cb_params2['depth']=9
        cb_params2['random_strength']=0.3
        cb_params2['one_hot_max_size']=10
        cb_params2['max_bin']=100
        cb_params2['l2_leaf_reg']=0.4189770851283918
#         cb_params2['bootstrap_type']='Bernoulli'

        cb_params3={
            'iterations': self.n_estimators,
            'random_strength': 0.1,
            'one_hot_max_size': 70, 'max_bin': 100,
            'learning_rate': 0.008,
            'l2_leaf_reg': 0.3,
            'grow_policy': 'Depthwise',
            'depth': 9,
            'max_bin': 200,
            'od_wait': 65,
            'bootstrap_type': 'Bayesian',
            'od_type': 'Iter',
            'eval_metric': 'Accuracy',
            'loss_function': 'MultiClass',
            'task_type': self.device.upper(),
            'random_state': self.random_state,
        }
        cb_params4=  cb_params.copy()
        cb_params4['learning_rate']=0.14300000000000002
        cb_params4['depth']=16
        cb_params4['random_strength']=0.5959959004533222
        cb_params4['one_hot_max_size']=100
        cb_params4['max_bin']=150
        cb_params4['l2_leaf_reg']=0.3838383515991557
        cb_params4['grow_policy']='Lossguide'

        dt_params= {'criterion': 'gini', 'max_depth': 9, 'min_samples_split': 17, 'min_samples_leaf': 18, 'max_features': 0.8433062331953287}
        etr_params= {'criterion': 'gini', 'max_depth': 16, 'min_samples_split': 11, 'min_samples_leaf': 1, 'max_features': 0.6678697051089325, 'bootstrap': True}
        hist_params= {'learning_rate': 0.05835392348975443, 'n_iter_no_change': 795, 'max_depth': 4, 'min_samples_leaf': 17, 'max_leaf_nodes': 98, 'l2_regularization': 1.923251027172902e-07}
        rf_params={'max_depth': 16, 'min_samples_split': 18, 'min_samples_leaf': 2, 'max_features': 0.4156475826924276}
        gbt_params={'learning_rate': 0.13640097404424165, 'max_depth': 7, 'min_samples_split': 17, 'min_samples_leaf': 15, 'subsample': 0.8858460576511749, 'max_features': 0.6114927460685724}
        knn_params={'n_neighbors': 16, 'weights': 'uniform', 'p': 2, 'leaf_size': 13, 'algorithm': 'ball_tree'}
        adb_params={'n_estimators': 957, 'learning_rate': 0.6625214812098597}

        models = {
###########             'svc': SVC(gamma="auto", probability=True, random_state=self.random_state),
            'xgb':  xgb.XGBClassifier(**xgb_params),
#             'xgb2': xgb.XGBClassifier(**xgb_params2),
#             'xgb3': xgb.XGBClassifier(**xgb_params3),
            'xgb4': xgb.XGBClassifier(**xgb_params4),
#             'xgb5': xgb.XGBClassifier(**xgb_params5),
           'lgb':  lgb.LGBMClassifier(**lgb_params),
            'lgb2': lgb.LGBMClassifier(**lgb_params2),
#             'lgb3': lgb.LGBMClassifier(**lgb_params3),
#             'lgb4': lgb.LGBMClassifier(**lgb_params4),
           'lgb5': lgb.LGBMClassifier(**lgb_params5),
#             'cat':  CatBoostClassifier(**cb_params),
#             'cat2': CatBoostClassifier(**cb_params2),
#             'cat3': CatBoostClassifier(**cb_params3),
#             'cat4': CatBoostClassifier(**cb_params4),
            "cat_sym": CatBoostClassifier(**cb_sym_params),
#             "cat_loss": CatBoostClassifier(**cb_loss_params),
#             'brf': BalancedRandomForestClassifier(n_estimators=4000,max_depth=12 ,n_jobs=-1, random_state=self.random_state),
            'rf': RandomForestClassifier(n_estimators=250,**rf_params, random_state=self.random_state),
            'hist_gbm' : HistGradientBoostingClassifier(max_iter=self.n_estimators,**hist_params,class_weight=class_weights_dict, random_state=self.random_state),
            'gbdt': GradientBoostingClassifier(**gbt_params,n_estimators=100,random_state=self.random_state),
            'ada': AdaBoostClassifier(**adb_params,random_state=self.random_state),
            'etr': ExtraTreesClassifier(**etr_params,random_state=self.random_state),
            'dt' : DecisionTreeClassifier(**dt_params,random_state=self.random_state),
#             'knn': KNeighborsClassifier(**knn_params),
            'log_reg': LogisticRegression(multi_class = 'ovr', max_iter = 1000),
            'ridge': CalibratedClassifierCV(RidgeClassifierCV(alphas=100.02351202464449), method='sigmoid'),
            'elasticNet':LogisticRegressionCV(Cs= [0.04426763148249415], l1_ratios= [0.9771433916131904]),
#             'ann1':init_ann1(3, X_test.shape[1]),
            'ann2':init_ann2(3, X_test.shape[1]),
#             'ann3':init_ann3(3, X_test.shape[1]),
            'ann4':init_ann4(3, X_test.shape[1]),
#             'ann5':init_ann5(3, X_test.shape[1]),


        }
        return models

In [ ]:
class OptunaWeights:
    def __init__(self, random_state, n_trials=3000):
        self.study = None
        self.weights = None
        self.random_state = random_state
        self.n_trials = n_trials

    def _objective(self, trial, y_true, y_preds):
        # Define the weights for the predictions from each model
        weights = [trial.suggest_float(f"weight{n}", 0, 1) for n in range(len(y_preds))]

        # Calculate the weighted prediction
        weighted_pred = np.average(np.array(y_preds), axis=0, weights=weights)
        weighted_pred = weighted_pred/weighted_pred.sum(axis=1, keepdims=True)

        weighted_pred_labels = np.argmax(weighted_pred, axis=1)
        accuracy = accuracy_score(y_true, weighted_pred_labels)

        log_loss_score=log_loss(y_true, weighted_pred)
        return accuracy

    def fit(self, y_true, y_preds):
        optuna.logging.set_verbosity(optuna.logging.ERROR)
        sampler = optuna.samplers.CmaEsSampler(seed=self.random_state)
        pruner = optuna.pruners.HyperbandPruner()
        self.study = optuna.create_study(sampler=sampler, pruner=pruner, study_name="OptunaWeights", direction='maximize')
        objective_partial = partial(self._objective, y_true=y_true, y_preds=y_preds)
        self.study.optimize(objective_partial, n_trials=self.n_trials)
        self.weights = [self.study.best_params[f"weight{n}"] for n in range(len(y_preds))]

    def predict(self, y_preds):
        assert self.weights is not None, 'OptunaWeights error, must be fitted before predict'
        weighted_pred = np.average(np.array(y_preds), axis=0, weights=self.weights)
        return weighted_pred

    def fit_predict(self, y_true, y_preds):
        self.fit(y_true, y_preds)
        return self.predict(y_preds)

    def weights(self):
        return self.weights

In [ ]:
import time


kfold = True
n_splits = 1 if not kfold else 6
random_state = 42
random_state_list = [42]
n_estimators = 9999
early_stopping_rounds = 600
verbose = False
splitter = Splitter(kfold=kfold, n_splits=n_splits)

# Initialize an array for storing test predictions
test_predss = np.zeros((X_test.shape[0], 3))
ensemble_score = []
ensemble_acc_score = []
weights = []
trained_models = {'xgb':[]}

for i, (X_train_, X_val, y_train_, y_val) in enumerate(splitter.split_data(X_train, y_train, random_state_list=random_state_list)):
    n = i % n_splits
    m = i // n_splits

    # Get a set of Regressor models
    classifier = Classifier(n_estimators, device, random_state)
    models = classifier.models

    # Initialize lists to store oof and test predictions for each base model
    oof_preds = []
    test_preds = []
    start_time_fold = time.time()

    # Loop over each base model and fit it to the training data, evaluate on validation data, and store predictions
    for name, model in models.items():
        start_time = time.time()
        if ('xgb' in name) or ('lgb' in name) or ('cat' in name)  :
            if 'lgb' in name:
                model.fit(X_train_, encode(y_train_,target_map), eval_set=[(X_val, encode(y_val,target_map))])
            else:
                model.fit(X_train_, encode(y_train_,target_map), eval_set=[(X_val, encode(y_val,target_map))], early_stopping_rounds=early_stopping_rounds, verbose=verbose)
        elif 'ann' in name:
            model.fit(X_train_, to_categorical(np.array(encode(y_train_,target_map)),3),\
                      validation_data=(X_val, to_categorical(np.array(encode(y_val,target_map)))),batch_size=10, epochs=50,verbose=verbose)#validation_data=(X_val, y_val)
        else:
            model.fit(X_train_, encode(y_train_,target_map))

        if name in trained_models.keys():
            trained_models[f'{name}'].append(deepcopy(model))

        if 'ann' in name:
            y_val_pred = np.array(model.predict(X_val))#[:,0]
            test_pred = np.array(model.predict(X_test))
        else:
            test_pred = model.predict_proba(X_test)
            y_val_pred = model.predict_proba(X_val)

        end_time = time.time()
        time_taken = end_time - start_time

        y_val_pred_labels = np.argmax(y_val_pred, axis=1)
        accuracy = accuracy_score(encode(y_val,target_map), y_val_pred_labels)

        score = log_loss(encode(y_val,target_map), y_val_pred)
        print(f'{name} [FOLD-{n} SEED-{random_state_list[m]}] Accuracy Score: {accuracy:.5f}, time taken :{time_taken:.3f} secs')

        oof_preds.append(y_val_pred)
        test_preds.append(test_pred)


    # Use Optuna to find the best ensemble weights
    optweights = OptunaWeights(random_state=random_state)
    y_val_pred = optweights.fit_predict(encode(y_val,target_map), oof_preds)

    score = log_loss(encode(y_val,target_map), y_val_pred)
    y_val_pred_labels = np.argmax(y_val_pred, axis=1)
    accuracy = accuracy_score(encode(y_val,target_map), y_val_pred_labels)

    end_time_fold = time.time()
    time_taken = end_time_fold - start_time_fold

    print(f'Ensemble [FOLD-{n} SEED-{random_state_list[m]}] -------------------> Accuracy Score: {accuracy:.5f}, Logloss: {score:.5f}, fold time taken :{time_taken:.5f} secs')

    ensemble_score.append(score)
    ensemble_acc_score.append(accuracy)
    weights.append(optweights.weights)

    # Predict to X_test by the best ensemble weights
    _test_preds = optweights.predict(test_preds)
    test_predss += _test_preds / (n_splits * len(random_state_list))

    gc.collect()


In [ ]:
# Calculate the mean LogLoss score of the ensemble
mean_score = np.mean(ensemble_acc_score)
std_score = np.std(ensemble_acc_score)
print(f'Ensemble Accuracy score {mean_score:.5f} ± {std_score:.5f}')

# Print the mean and standard deviation of the ensemble weights for each model
print('--- Model Weights ---')
mean_weights = np.mean(weights, axis=0)
std_weights = np.std(weights, axis=0)
for name, mean_weight, std_weight in zip(models.keys(), mean_weights, std_weights):
    print(f'{name}: {mean_weight:.5f} ± {std_weight:.5f}')

In [ ]:
def visualize_importance(models, feature_cols, title, head=15):
    importances = []
    feature_importance = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df["importance"] = model.feature_importances_
        _df["feature"] = pd.Series(feature_cols)
        _df["fold"] = i
        _df = _df.sort_values('importance', ascending=False)
        _df = _df.head(head)
        feature_importance = pd.concat([feature_importance, _df], axis=0, ignore_index=True)

    feature_importance = feature_importance.sort_values('importance', ascending=False)
    # display(feature_importance.groupby(["feature"]).mean().reset_index().drop('fold', axis=1))
    plt.figure(figsize=(18, 10))
    sns.barplot(x='importance', y='feature', data=feature_importance, color= (0.8, 0.5, 0.3), errorbar='sd')
    plt.xlabel('Importance', fontsize=14)
    plt.ylabel('Feature', fontsize=14)
    plt.title(f'{title} Feature Importance', fontsize=18)
    plt.grid(True, axis='x')
    plt.show()

for name, models in trained_models.items():
    visualize_importance(models, list(X_train.columns), name)

In [ ]:
submission[target] =   decode(np.argmax(test_predss, axis=1),target_map)
submission.to_csv('submission_pure.csv',index=False)
submission.head()

### 2

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline
plt.style.use('fivethirtyeight')

import optuna

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier, VotingClassifier, RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_validate, cross_val_predict, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SequentialFeatureSelector

In [ ]:
train_df=pd.read_csv("/kaggle/input/playground-series-s4e6/train.csv")
test_df=pd.read_csv('/kaggle/input/playground-series-s4e6/test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.duplicated().sum()

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
init_features = list(train_df.columns)
init_features

In [ ]:
for col in train_df.columns:
    print(f'{col} has {train_df[col].nunique()} unique values')

In [ ]:
plt.figure(figsize=(12, 10))
ax = sns.countplot(x='Target', data=train_df)

# Add labels to each bar in the plot
for p in ax.patches:
    ax.text(p.get_x() + p.get_width() / 2, p.get_height() + 3, f'{int(p.get_height())}', ha="center")

plt.xlabel('Target Variable')
plt.ylabel('Count')
plt.title('Target Variable Distribution')
plt.show()